In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv')
df.head()

,id,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,1,Vistara,UK-810,Bangalore,Early_Morning,one,Night,Mumbai,Economy,14.25,21,7212
1,2,SpiceJet,SG-5094,Hyderabad,Evening,zero,Night,Kolkata,Economy,1.75,7,5292
2,3,Vistara,UK-846,Bangalore,Morning,one,Evening,Delhi,Business,9.58,5,60553
3,4,Vistara,UK-706,Kolkata,Morning,one,Evening,Hyderabad,Economy,6.75,28,5760
4,5,Indigo,6E-5394,Chennai,Early_Morning,zero,Morning,Mumbai,Economy,2.00,4,10712


In [3]:
def prepare_df(df):
  df[['flight_country', 'flight_nummber']]= df['flight'].str.split("-", expand=True)
  df['flight_number'] = df['flight_nummber'].astype(int)

  stops = {'one': 0, 'zero': 1, 'two_or_more': 2}
  classes = {'Economy': 0, 'Business': 1}
  df['stops'] = df['stops'].replace(stops)
  df['class'] = df['class'].replace(classes)

  encoder = OrdinalEncoder()
  df['flight_country'] = encoder.fit_transform(df[['flight_country']])

  one_hot_encoder = ['airline', 'source_city', 'departure_time', 'arrival_time','destination_city']
  df = pd.get_dummies(df, columns=one_hot_encoder)

  df = df.replace({True: 1, False: 0})

  df.drop(['id', 'flight'], axis=1, inplace=True)

  return df

In [4]:
df = prepare_df(df)

/tmp/ipykernel_17/3659705708.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['stops'] = df['stops'].replace(stops)
/tmp/ipykernel_17/3659705708.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['class'] = df['class'].replace(classes)
/tmp/ipykernel_17/3659705708.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent

In [5]:
df.head()
df.shape

(20000, 38)

Preparing our X set and Y set to train# 

In [6]:
X = df.drop('price', axis=1)
Y = df['price']

In [7]:
model = RandomForestRegressor()
model.fit(X,Y)

RandomForestRegressor()

In [8]:
test = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv')
test = prepare_df(test)
test.head()

/tmp/ipykernel_17/3659705708.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['stops'] = df['stops'].replace(stops)
/tmp/ipykernel_17/3659705708.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['class'] = df['class'].replace(classes)
/tmp/ipykernel_17/3659705708.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent

,stops,class,duration,days_left,flight_country,flight_nummber,flight_number,airline_AirAsia,airline_Air_India,airline_GO_FIRST,...,arrival_time_Evening,arrival_time_Late_Night,arrival_time_Morning,arrival_time_Night,destination_city_Bangalore,destination_city_Chennai,destination_city_Delhi,destination_city_Hyderabad,destination_city_Kolkata,destination_city_Mumbai
0,0,1,28.25,2,1.0,765,765,0,1,0,...,0,0,0,1,0,0,1,0,0,0
1,0,1,13.83,34,5.0,747,747,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1,1,2.00,30,1.0,570,570,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,5.17,26,3.0,974,974,1,0,0,...,0,1,0,0,0,0,1,0,0,0
4,0,0,16.33,35,1.0,770,770,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
predictions = model.predict(test)

In [10]:
submission = pd.DataFrame({'id': range(len(predictions)), 'price': predictions})
submission.to_csv('submission.csv', index=False)

In [11]:
submission = pd.read_csv('/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv')
submission.price = predictions
submission.to_csv('submission.csv', index=False)

In [12]:
pd.read_csv('/kaggle/working/submission.csv').head()



,id,price
0,1,55798.56
1,2,63793.52
2,3,23399.44
3,4,2503.30
4,5,5665.10
